# Overview

The data has been split into two groups:
-training set (train.csv)
-test set (test.csv)

**The training set** should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

**The test set** should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include **gender_submission.csv**, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

## Data Dictionary

| Variable |                 Definition                 |                       Key                      |
|:--------:|:------------------------------------------:|:----------------------------------------------:|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

## Variable Notes

**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

## Link

Topic:
[Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic)

Helpers:
[YT random forest](https://www.youtube.com/watch?v=fATVVQfFyU0)
[Kaggle tutorial](https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy)
[Kaggle plots examples](https://www.kaggle.com/code/andrej0marinchenko/titanic-machine-learning-from-disaster/)
[Udemy with love](https://www.udemy.com/course/machinelearning/)

In [ ]:
# do operacji na tablicach
import pandas as pd
import numpy as np

# do splitera danych na zbiór testowy i walidacyjny
from sklearn.model_selection import StratifiedShuffleSplit

# do obróbki wykresów
from sklearn.tree import plot_tree

# do rysowania heatmapy
import seaborn as sns

# to do podmiany NaN na coś innego
from sklearn.impute import SimpleImputer

# do to rozdzielenia właściwości na inne kolumn
from sklearn.preprocessing import OneHotEncoder

# do pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

# żeby był scaler
from sklearn.preprocessing import StandardScaler

# do random forestu
from sklearn.ensemble import RandomForestClassifier
# do cross validacji
from sklearn.model_selection import GridSearchCV

# do confusion matrix żęby zwizualizować wyniki ze zbioru testowego
from sklearn.metrics import confusion_matrix

# do decision tree
from sklearn.tree import DecisionTreeClassifier

# do narysowania drzewa decyzyjnego
import matplotlib.pyplot as plt

# do logistic regression
from sklearn.linear_model import LogisticRegression

# do XGBoost
from xgboost import XGBClassifier

In [ ]:
titanic_train = pd.read_csv("train.csv")
titanic_train.head()

In [ ]:
titanic_train.info()

In [ ]:
# stworzenie heat mapy prezentującej korelacje między właściwościami
ax = plt.subplots(figsize =(14, 12))
cp = sns.color_palette("icefire", 18)
ax = sns.heatmap(titanic_train.corr(), cmap=cp, linecolor="white", linewidths=0.2, annot=True)
ax.hlines([1,2], *ax.get_xlim(), color= "green", linewidth=3)
ax.vlines([1,2], *ax.get_ylim(), color= "green", linewidth=3)

# Podział danych

Prawdopodobnie nie potrzebne w tym momencie - tylko informacyjnie i 'na zapas'
Wszystkie dane z niego są pominięte

In [ ]:
# tworze spliter, którym podziele dane
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
# n_splits - Number of re-shuffling & splitting iterations.
# test_size - represent the proportion of the dataset to include in the test split.

In [ ]:
# spliter zdefiniowany jako klasa żeby tego wszystkiego nie powtarzać pare razy w kolejnej komórce
class Spliter(StratifiedShuffleSplit):

    def splitBy(columns):
        print(columns)
        # dzieli dane tak aby proporcje drugiej wartości były równe pomiędzy oba setami
        # po to by uniknać sytuacji (teoretycznie możliwej) że w train setcie będą tylko wartości że ktoś przeżył
        for train_index, test_index  in split.split(titanic_train, titanic_train[columns]):
            spl_train_set = titanic_train.loc[train_index]
            spl_test_set  = titanic_train.loc[test_index]
        return spl_train_set, spl_test_set


    # licze stosunek wartości umarł/przeżył między setami żeby wypisać żeby udowodnić że dzielenie przez konkretą kolumne ma sens
    def countRatio(spl_train_set, spl_test_set):
        c_spl_train_set = spl_train_set.groupby(["Survived"])["Survived"].count()
        c_spl_test_set = spl_test_set.groupby(["Survived"])["Survived"].count()
        train_ratio = c_spl_train_set[0] / c_spl_train_set[1]
        test_ratio = c_spl_test_set[0] / c_spl_test_set[1]
        print("Train ratio: " + str(train_ratio))
        print("Test ratio: " + str(test_ratio))

In [ ]:
# wypisuje na ekran stosunki przy różnych podziałach - jako dowód że spliter miał sens
spl_train_set, spl_test_set = Spliter.splitBy(["Pclass"])
Spliter.countRatio(spl_train_set, spl_test_set)

spl_train_set, spl_test_set = Spliter.splitBy(["Pclass","Survived"])
Spliter.countRatio(spl_train_set, spl_test_set)

spl_train_set, spl_test_set = Spliter.splitBy(["Survived"])
Spliter.countRatio(spl_train_set, spl_test_set)

# Obróbka danych

In [ ]:
# musimy usupełnić puste dane w kolumnach np NaN w Age
# robimy to teraz a nie przed podziałem bo nie chcemy wpływać na dane zanim będą rozdzielone bo możemy wpłynąć na wynik końcowy
# powiedzmy że wstawiamy średni wiek do wszystkich kolumn gdzie jest NaN a średni wiek będzie inny w w zależności od tego kiedy dokonaliśmy podziału

In [ ]:
# zamiast ręcznie wszystko wywoływać za każdym razem tworzymy pipeline tzn taki ciąg zdarzeń
# zdefiniujemy pare klas, które będą modyfikować dataSet

In [ ]:
class AgeImputer(TransformerMixin):

    # tej metody wymaga pipeline
    def fit(self, dataFrame):
        return self

    # ta metoda tak nazwana bo pipeline ją wywołuje
    def transform(self, dataFrame):
        # imputer który uzupełni dane średnią wszystkich innych wartości
        imputer = SimpleImputer(strategy="mean")
        dataFrame['Age'] = imputer.fit_transform(dataFrame[['Age']])
        return dataFrame


In [ ]:
# kolumny tekstowe muszą być enkodowane bo w ML nie używamy tekstu tylko liczby
# więc musimy np C -> 0, S -> 1
# to teoretycznie by wystarczyło ale może prowadzić do błędnych predykcji
# model może szukać zależności między wierszami i jak zobaczyłby pattern to może wpłynąć na jego predykcje
# np jakby wiersze były by w kolejności żę by wychodziło 0 1 2 0 1 2 0 1 2 to może to powiązać z passangerId i prowadzić do złych predykcji
# dlatego wartości C S itd rozdzielamy do osobnych kolumn

In [ ]:
class FeatureEncoder(TransformerMixin) :

    def fit(self, dataFrame):
        return self

    def transform(self, dataFrame):
        encoder = OneHotEncoder()
        dataFrame = self.transformEmbarked(encoder, dataFrame)
        dataFrame = self.transformAge(encoder, dataFrame)
        return dataFrame

    def transformEmbarked(self, encoder, dataFrame):
        # fituje encoder z kolumną danych
        matrix = encoder.fit_transform(dataFrame[['Embarked']]).toarray()

        column_names = dataFrame['Embarked'].unique()
        # print(column_names)

        # tym .T obracamy tabele (zamiana x z y) żeby łatwo wymieniać wartości
        for i in range(len(matrix.T)):
            dataFrame[column_names[i]] = matrix.T[i]
        return dataFrame

    def transformAge(self, encoder, dataFrame):
        matrix = encoder.fit_transform(dataFrame[['Sex']]).toarray()

        column_names = dataFrame['Sex'].unique()

        for i in range(len(matrix.T)):
            dataFrame[column_names[i]] = matrix.T[i]

        return dataFrame

In [ ]:
class ColumnsDropper(TransformerMixin):

    def fit(self, dataFrame):
        return self

    def transform(self, dataFrame):
        print(dataFrame)

        # zostawiam:
            # PassengerId - żeby wiedzieć kto jest kim
            # Survived - czy umarł czy nie
            # Pclass - im lepsza klasa tym pewnie miał bliżej do szalup i go lepiej traktowano
            # SibSp, Parch - bo może trzymali się razem albo ratowali się wzajemnie
            # rozdzielone Sex - bo kobiety i dzieci przodem
            # rozdzielone Embarked - im później wsiedli tym pewnie mięli kabiny bliżej wyjścia

        # usuwam:
            # Ticket - bo randomowa nazwa
            # Name - nie zauważono korelacji imienia ze śmiertelnością
            # Cabin - brakuje mega wiele danych tutaj - titanic był podzielone na strefy i w nazwie kabiny jest strefa, mając pełne dane i wyciągając literę z nazwy model byłby dokładniejszy ale brakuje większość danych
            # Embarked - rozdzielamy to na osobne kolumny
            # Sex - rozdzielamy to na osobne kolumny
            # Fare - opłata za bilet wpływa na klasę ale nie wpływa na umieralność
            # np.nan - podczas użycia dataFrame['Embarked'].unique() jeżeli istniały kolumny puste tzn z NaN to zostały dodane kolumny z nazwą "NaN". To nie jest nazwa testowa tylko taka 'pusta'
            # nie możemy jej usunać wpisująć jej nazwę "NaN" tylko musimy dosłownie powiedzieć że jest to puste wiec np.nan
        # może się okazać, że w ogóle nie było jakieś tej kolumny to olewamy errors=ignore
        return dataFrame.drop(["Ticket", "Name", "Cabin", "Embarked", "Sex", np.nan, "Fare"], axis=1, errors="ignore")


In [ ]:
# tworzymy pipeline przez który mozna przepuścić dane i zostaną obropione po kolei przez każdą klasę
pipeline = Pipeline([("AgeImputer", AgeImputer()),
                     ("FeatureEncoder", FeatureEncoder()),
                     ("ColumnsDropper", ColumnsDropper())])

In [ ]:
titanic_train.head()

In [ ]:
# transofmracja danych treningowych

transformed_train_set = pipeline.fit_transform(titanic_train)
transformed_train_set.head()

In [ ]:
ax = plt.subplots(figsize =(14, 12))
ax = sns.heatmap(transformed_train_set.corr(), cmap=cp, linecolor="white", linewidths=0.2)
ax.hlines([1, 2], *ax.get_xlim(), color="green", linewidth=3)
ax.vlines([1, 2], *ax.get_ylim(), color="green", linewidth=3)

In [ ]:
# transofmracja danych testowych
# tutaj żeby rozdzielić 'moduły' dokumentu

titanic_test = pd.read_csv("test.csv")
transformed_test_set = pipeline.fit_transform(titanic_test)
# fillna wypełnia NA/NaN wartościami według metody: ffill -> propaguj ostatnią ważną obserwację do następnego ważnego uzupełnienia

transformed_test_set = transformed_test_set.fillna(method="ffill")
transformed_test_set.head()

In [ ]:
# Standaryzacja danych do modeli machine learning polega na przekształceniu danych pierwotnych, aby ich rozkład miał średnią wartość równą 0 i odchylenie standardowe równe 1. Od każdej wartość w kolumnie danych będzie odejmowana średnia wartość kolumny, a następnie to co wyjdzie będzie podzielona przez odchylenie standardowe kolumny danych. Opisany proces dotyczy każdej kolumny oddzielnie.
# http://sigmaquality.pl/uncategorized/standaryzacja-danych-do-modeli-machine-learning-za-pomoca-standardscaler/
# w skrócie zmienia zwykłe dane w -1.51772497e+00  8.20947424e-01 -6.31268373e-01

In [ ]:
class ScalerSpliter():

    scaler = StandardScaler()

    def fitAndSplit(self, dataFrame):
        # dzielimy trainset -> właściwości / wyniki
        X = dataFrame.drop(['Survived'], axis=1)
        Y = dataFrame['Survived']

        # dopasowywujemy fit'er do danych
        X_data = self.scaler.fit_transform(X)
        # musimy to przekonwertować do numpy arraya
        Y_data = Y.to_numpy()

        return X_data, Y_data

    def transform(self, dataFrame) :
        return self.scaler.transform(dataFrame)

In [ ]:
# stworze tutaj obiekt raz i będę go używać też w modelu testowym
# scaler fitujemy do traningowego setu i wykorzystujemy go później w testowym - przez wszystkie uśrednienia i dopasowania
spliter = ScalerSpliter()

# Metody predykcji

## Random forest

In [ ]:
X_train_forest, Y_train_forest = spliter.fitAndSplit(transformed_train_set)

In [ ]:
# w zależności od hiperparametrów model będzie miał różne wyniki
# to pomoże nam zdecydować, jakie parametry są najlepsze
class EstimatorFinder:
    def getBestEstimator(X_train, Y_train):
        clf = RandomForestClassifier()

        # parametry random forestu
        # będą wypróbowane wszystkie w różnych kombinacjach np 10 None 2 / 10 None 3 / 10 None 4 itd
        param_grid = [
            {"n_estimators": [10, 100, 200, 500],"max_depth": [None, 5, 10], "min_samples_split": [2,3,4]}
        ]

        # cross validation tzn program dzieli dataSet na np 10 części
        # 9 częściami trenuje dane, 1 testuje
        # przerabiam wszystkie kombinacje tak żeby każdy 'fold'/część była użyta raz jako walidacja
        # i tak GridSearchCV znajduje optymalną kombinacje hiperparametrów
        # parametr - cv to ile foldów
        grid_search = GridSearchCV(clf, param_grid, cv=3, scoring="accuracy", return_train_score=True)
        grid_search.fit(X_train, Y_train)
        return grid_search.best_estimator_


In [ ]:
forest_clf = EstimatorFinder.getBestEstimator(X_train_forest, Y_train_forest)
forest_clf

In [ ]:
# teraz sprawdzenie na danych testowych jaka jest dokładność

In [ ]:
X_test_forest = spliter.transform(transformed_test_set)
forest_predictions = forest_clf.predict(X_test_forest)

In [ ]:
# otrzymamy tylko predykcje tak więc musimy 'odtworzyć' pełną tabele
final_forest_table = pd.DataFrame(titanic_test['PassengerId'])
final_forest_table['Survived'] = forest_predictions
final_forest_table

In [ ]:
# zapis do pliku <- i to wysyłamy do kaggla
final_forest_table.to_csv("random_forest.csv", index=False)

In [ ]:
# trochę statystyk do zbioru testowego na podstawie gotowego random forestu

In [ ]:
forest_train_set_predictions = forest_clf.predict(X_train_forest)

# confusion matrix na podstawie danych testowych
confusion_matrix(Y_train_forest, forest_train_set_predictions)

In [ ]:
# wynik modelu - podliczone to co w confusion matrixie
forest_clf.score(X_train_forest, Y_train_forest)

# Decision tree

In [ ]:
X_train_tree, Y_train_tree = spliter.fitAndSplit(transformed_train_set)

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth = 10, random_state = 0)
tree_clf.fit(X_train_tree, Y_train_tree)

In [ ]:
X_test_tree = spliter.transform(transformed_test_set)

In [ ]:
tree_predictions = tree_clf.predict(X_test_tree)

In [ ]:
final_tree_table = pd.DataFrame(titanic_test['PassengerId'])
final_tree_table['Survived'] = tree_predictions
final_tree_table

In [ ]:
plt.subplots(figsize=(20, 16))
plot_tree(tree_clf, filled=True)

In [ ]:
# zapis do pliku <- i to wysyłamy do kaggla
final_forest_table.to_csv("tree.csv", index=False)

In [ ]:
# trochę statystyk do zbioru testowego na podstawie gotowego drzewa decyzyjnego

In [ ]:
tree_predictions_train = tree_clf.predict(X_train_tree)
confusion_matrix(Y_train_tree, tree_predictions_train)

In [ ]:
tree_clf.score(X_train_tree, Y_train_tree)

# Logistic regression

In [ ]:
X_train_log, Y_train_log = spliter.fitAndSplit(transformed_train_set)

In [ ]:
log_clf = LogisticRegression(random_state=0, max_iter=1000)
log_clf.fit(X_train_log, Y_train_log)

In [ ]:
X_test_log = spliter.transform(transformed_test_set)

In [ ]:
log_predictions = log_clf.predict(X_test_tree)

In [ ]:
final_log_table = pd.DataFrame(titanic_test['PassengerId'])
final_log_table['Survived'] = log_predictions
final_log_table

In [ ]:
# zapis do pliku <- i to wysyłamy do kaggla
final_log_table.to_csv("reg_log.csv", index=False)

In [ ]:
# trochę statystyk do zbioru testowego na podstawie gotowego logistic regression

In [ ]:
log_predictions_train = log_clf.predict(X_train_tree)
confusion_matrix(Y_train_log, log_predictions_train)

In [ ]:
log_clf.score(X_train_log, Y_train_log)

# XGB Classifier

In [ ]:
X_train_xgb, Y_train_xgb = spliter.fitAndSplit(transformed_train_set)

In [ ]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train_xgb, Y_train_xgb)

In [ ]:
X_test_xgb = spliter.transform(transformed_test_set)

In [ ]:
xgb_predictions = xgb_clf.predict(X_test_xgb)

In [ ]:
final_xgb_table = pd.DataFrame(titanic_test['PassengerId'])
final_xgb_table['Survived'] = xgb_predictions
final_xgb_table

In [ ]:
# zapis do pliku <- i to wysyłamy do kaggla
final_xgb_table.to_csv("xgb.csv", index=False)

In [ ]:
# trochę statystyk do zbioru testowego na podstawie gotowego xbt

In [ ]:
xgb_clf.score(X_train_xgb, Y_train_xgb)

In [ ]:
xgb_predictions = xgb_clf.predict(X_train_xgb)

In [ ]:
confusion_matrix(Y_train_xgb, xgb_predictions)